Check for all operators in script block of a process:

In [338]:
import os
import re
import csv
import pandas as pd

def find_operators_in_script(directory, pattern_list, output_file_name):
    result_list = []

    for pipeline in os.listdir(directory):
        subdirectory_path = os.path.join(directory, pipeline)
        folder_name = os.path.basename(subdirectory_path)
        #print(pipeline)

        for root, dirs, files in os.walk(subdirectory_path):
            for filename in files:
                file_path = os.path.join(root, filename)
                
                if filename.endswith('.nf') or filename.endswith('.config'):
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                        lines = file.readlines()
                    
                    inside_block = False
                    brace_stack = []
                    process_flag = False
                    process_name = None

                    process_pattern = r'process\s+(\w+)\s*{'
                    
                    for line_num, line in enumerate(lines):
                        stripped_line = line.strip()
                        
                        if line.strip().startswith(('#', '//', '*')):
                            continue

                        if re.search(process_pattern, stripped_line):
                            process_flag = True
                            process_match = re.search(process_pattern, stripped_line)
                            if process_match:
                                process_name = process_match.group(1)
                            brace_stack.append('{')

                        if '{' in stripped_line:
                            brace_stack.append('{')

                        if '}' in stripped_line:
                            if brace_stack:
                                brace_stack.pop()
                                if not brace_stack and process_flag:
                                    process_flag = False

                        if '"""' in stripped_line:
                            inside_block = not inside_block

                        if process_flag and inside_block:
                            operator_pattern = "(" + "|".join(map(re.escape, pattern_list)) + ")"
                            if re.search(operator_pattern, stripped_line):
                                for pattern in pattern_list:
                                    if re.search(re.escape(pattern), stripped_line):
                                        result_list.append([folder_name, process_name, pattern, file_path, line_num+1, stripped_line])

    write_to_csv(result_list, output_file_name)

def write_to_csv(result_list, output_file_name):
    with open(f'./results/{output_file_name}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['pipeline', 'process_name', 'operator', 'file_path', 'line_number', 'line_content'])
        writer.writerows(result_list)

In [339]:
'''directory_path = './git_repos'

df_operators = pd.read_excel('./data/all_operator_dataset.xlsx')
operator_list = [str(operator) for operator in df_operators['operator']]

output_file_name = 'all_operators_in_scripts_new'

find_operators_in_script(directory_path, operator_list, output_file_name)'''

"directory_path = './git_repos'\n\ndf_operators = pd.read_excel('./data/all_operator_dataset.xlsx')\noperator_list = [str(operator) for operator in df_operators['operator']]\n\noutput_file_name = 'all_operators_in_scripts_new'\n\nfind_operators_in_script(directory_path, operator_list, output_file_name)"

In [340]:
'''directory_path = './git_repos'

df_operators = pd.read_excel('./data/bioinformatics_tools.xlsx')
operator_list = [str(operator) for operator in df_operators['operator']]

output_file_name = 'bioinformatic_tools_in_scripts'

find_operators_in_script(directory_path, operator_list, output_file_name)'''

In [42]:
import pandas as pd

bioinformatics_script = pd.read_csv('./results/bioinformatic_tools_in_scripts.csv')
bioinformatics_script

,pipeline,process_name,operator,file_path,line_number,line_content
0,airrflow,FASTQC_POSTASSEMBLY,fastqc,./git_repos\airrflow\modules\local\fastqc_postassembly.nf,24,fastqc $args --threads $task.cpus ${prefix}_ASSEMBLED.fastq
1,airrflow,FASTQC_POSTASSEMBLY,fastqc,./git_repos\airrflow\modules\local\fastqc_postassembly.nf,28,"fastqc: \$( fastqc --version | sed -e ""s/FastQC v//g"" )"
2,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,39,fastp \\
3,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,43,--json ${prefix}.fastp.json \\
4,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,44,--html ${prefix}.fastp.html \\
5,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,48,2> ${prefix}.fastp.log \\
6,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,49,| gzip -c > ${prefix}.fastp.fastq.gz
7,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,53,"fastp: \$(fastp --version 2>&1 | sed -e ""s/fastp //g"")"
8,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,60,fastp \\
9,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,62,--out1 ${prefix}.fastp.fastq.gz \\


In [43]:
all_tools = bioinformatics_script['operator'].value_counts()
all_tools

operator
samtools                         1098
multiqc                           447
bwa                               385
fastp                             348
gatk                              336
fastqc                            236
bowtie2                           169
bedtools                          154
bcftools                          125
picard                            122
qiime                              87
cutadapt                           69
kraken2                            66
tabix                              63
salmon                             55
trim_galore                        54
hisat2                             52
bwa-mem2                           44
minimap2                           31
featureCounts                      29
gffread                            29
stringtie                          28
seqtk                              26
gtdbtk                             22
bowtie2-build                      22
seqkit                             20
mac

Add assigned Group == 0 

In [44]:
assigned_group = 0
bioinformatics_script['Assigned Group'] = assigned_group

In [45]:
bioinformatics_script = bioinformatics_script.rename(columns={'Operator': 'operator'})
bioinformatics_script.head()

,pipeline,process_name,operator,file_path,line_number,line_content,Assigned Group
0,airrflow,FASTQC_POSTASSEMBLY,fastqc,./git_repos\airrflow\modules\local\fastqc_postassembly.nf,24,fastqc $args --threads $task.cpus ${prefix}_ASSEMBLED.fastq,0
1,airrflow,FASTQC_POSTASSEMBLY,fastqc,./git_repos\airrflow\modules\local\fastqc_postassembly.nf,28,"fastqc: \$( fastqc --version | sed -e ""s/FastQC v//g"" )",0
2,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,39,fastp \\,0
3,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,43,--json ${prefix}.fastp.json \\,0
4,airrflow,FASTP,fastp,./git_repos\airrflow\modules\nf-core\fastp\main.nf,44,--html ${prefix}.fastp.html \\,0


# Analysis of operators in operator domain dataset

In [143]:
import pandas as pd

operator_script = pd.read_csv('./results/all_operators_in_scripts_new.csv')
df_operator_clustered = pd.read_csv('./data/operator_dataset_clustered.csv')

In [144]:
df_operator_clustered_assigned_group_one = df_operator_clustered[df_operator_clustered['Assigned Group'] == 0]

In [145]:
df_operator_clustered_assigned_group_one

,Operator,Assigned Group
2,samtools,0
9,bcftools,0
22,bedtools,0
39,fastqc,0
46,bwa,0
47,bedextract,0
81,seqtk,0
83,{output.bed},0
99,{output.bam},0
112,{input.bam},0


In [146]:
df_operator_clustered.head()

,Operator,Assigned Group
0,{log},6
1,{output},6
2,samtools,0
3,echo,2
4,shell:,4


Add Assigned Group == 0  to bioinformatic tools

In [147]:
all_process = operator_script.groupby(['pipeline', 'process_name']).size().reset_index(name='operator_count')

In [148]:
sum_all_process = len(all_process)
sum_all_process

2941

In [187]:
operator_script = operator_script[operator_script['operator'].str.len() >= 4]
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
print(len(operator_script))
operator_script.head(600)

29532


,pipeline,process_name,operator,file_path,line_number,line_content
62,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,335,echo $workflow.manifest.version > v_pipeline.txt
72,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt
76,diaproteomics,get_software_versions,nextflow,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt
117,diaproteomics,dda_raw_file_conversion,Name,./git_repos/diaproteomics/main.nf,359,ThermoRawFileParser.sh -i=${raw_file} -f=2 -b=${raw_file.baseName}.mzML
119,diaproteomics,dda_raw_file_conversion,file,./git_repos/diaproteomics/main.nf,359,ThermoRawFileParser.sh -i=${raw_file} -f=2 -b=${raw_file.baseName}.mzML
129,diaproteomics,dda_id_format_conversion,FileConverter,./git_repos/diaproteomics/main.nf,380,IDFileConverter -in ${dda_id_file} -out ${id}_${sample}_peptide_ids.idXML -threads ${task.cpus}
132,diaproteomics,dda_id_format_conversion,file,./git_repos/diaproteomics/main.nf,380,IDFileConverter -in ${dda_id_file} -out ${id}_${sample}_peptide_ids.idXML -threads ${task.cpus}
135,diaproteomics,dda_library_generation,convert,./git_repos/diaproteomics/main.nf,402,easypqp convert \\
143,diaproteomics,dda_library_generation,file,./git_repos/diaproteomics/main.nf,403,--unimod ${unimod_file} \\
150,diaproteomics,dda_library_generation,file,./git_repos/diaproteomics/main.nf,404,--pepxml ${idxml_file} \\


In [188]:
operator_script.dropna(inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12680\2951588785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operator_script.dropna(inplace=True)


In [189]:
operator_script_total = operator_script

In [190]:
operator_script = operator_script[~operator_script['file_path'].str.endswith('.config')]
operator_script

,pipeline,process_name,operator,file_path,line_number,line_content
62,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,335,echo $workflow.manifest.version > v_pipeline.txt
72,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt
76,diaproteomics,get_software_versions,nextflow,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt
117,diaproteomics,dda_raw_file_conversion,Name,./git_repos/diaproteomics/main.nf,359,ThermoRawFileParser.sh -i=${raw_file} -f=2 -b=${raw_file.baseName}.mzML
119,diaproteomics,dda_raw_file_conversion,file,./git_repos/diaproteomics/main.nf,359,ThermoRawFileParser.sh -i=${raw_file} -f=2 -b=${raw_file.baseName}.mzML
129,diaproteomics,dda_id_format_conversion,FileConverter,./git_repos/diaproteomics/main.nf,380,IDFileConverter -in ${dda_id_file} -out ${id}_${sample}_peptide_ids.idXML -threads ${task.cpus}
132,diaproteomics,dda_id_format_conversion,file,./git_repos/diaproteomics/main.nf,380,IDFileConverter -in ${dda_id_file} -out ${id}_${sample}_peptide_ids.idXML -threads ${task.cpus}
135,diaproteomics,dda_library_generation,convert,./git_repos/diaproteomics/main.nf,402,easypqp convert \\
143,diaproteomics,dda_library_generation,file,./git_repos/diaproteomics/main.nf,403,--unimod ${unimod_file} \\
150,diaproteomics,dda_library_generation,file,./git_repos/diaproteomics/main.nf,404,--pepxml ${idxml_file} \\


In [191]:
operator_script_total

,pipeline,process_name,operator,file_path,line_number,line_content
62,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,335,echo $workflow.manifest.version > v_pipeline.txt
72,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt
76,diaproteomics,get_software_versions,nextflow,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt
117,diaproteomics,dda_raw_file_conversion,Name,./git_repos/diaproteomics/main.nf,359,ThermoRawFileParser.sh -i=${raw_file} -f=2 -b=${raw_file.baseName}.mzML
119,diaproteomics,dda_raw_file_conversion,file,./git_repos/diaproteomics/main.nf,359,ThermoRawFileParser.sh -i=${raw_file} -f=2 -b=${raw_file.baseName}.mzML
129,diaproteomics,dda_id_format_conversion,FileConverter,./git_repos/diaproteomics/main.nf,380,IDFileConverter -in ${dda_id_file} -out ${id}_${sample}_peptide_ids.idXML -threads ${task.cpus}
132,diaproteomics,dda_id_format_conversion,file,./git_repos/diaproteomics/main.nf,380,IDFileConverter -in ${dda_id_file} -out ${id}_${sample}_peptide_ids.idXML -threads ${task.cpus}
135,diaproteomics,dda_library_generation,convert,./git_repos/diaproteomics/main.nf,402,easypqp convert \\
143,diaproteomics,dda_library_generation,file,./git_repos/diaproteomics/main.nf,403,--unimod ${unimod_file} \\
150,diaproteomics,dda_library_generation,file,./git_repos/diaproteomics/main.nf,404,--pepxml ${idxml_file} \\


In [192]:
operators_in_all_process = operator_script.groupby(['pipeline', 'process_name']).size().reset_index(name='operator_count')
operators_in_all_process

,pipeline,process_name,operator_count
0,airrflow,ADD_META_TO_TAB,11
1,airrflow,AIRRFLOW_REPORT,21
2,airrflow,CHANGEO_ASSIGNGENES,4
3,airrflow,CHANGEO_CONVERTDB_FASTA,6
4,airrflow,CHANGEO_CREATEGERMLINES,4
5,airrflow,CHANGEO_MAKEDB,5
6,airrflow,CHANGEO_PARSEDB_SELECT,8
7,airrflow,CHANGEO_PARSEDB_SPLIT,3
8,airrflow,COLLAPSE_DUPLICATES,16
9,airrflow,DEFINE_CLONES,17


In [193]:
java = operator_script[operator_script['operator'] == 'java']
java

,pipeline,process_name,operator,file_path,line_number,line_content
13557,methylseq,QUALIMAP_BAMQC,java,./git_repos/methylseq/modules/nf-core/qualimap/bamqc/main.nf,38,export _JAVA_OPTIONS=-Djava.io.tmpdir=./tmp
13574,methylseq,QUALIMAP_BAMQC,java,./git_repos/methylseq/modules/nf-core/qualimap/bamqc/main.nf,40,--java-mem-size=$memory \\
14201,methylseq,QUALIMAP_BAMQCCRAM,java,./git_repos/methylseq/modules/nf-core/qualimap/bamqccram/main.nf,40,export _JAVA_OPTIONS=-Djava.io.tmpdir=./tmp
14229,methylseq,QUALIMAP_BAMQCCRAM,java,./git_repos/methylseq/modules/nf-core/qualimap/bamqccram/main.nf,44,--java-mem-size=$memory \\
27071,createpanelrefs,GATK4_PREPROCESSINTERVALS,java,./git_repos/createpanelrefs/modules/nf-core/gatk4/preprocessintervals/main.nf,38,"gatk --java-options ""-Xmx${avail_mem}M -XX:-UsePerfData"" \\"
27204,createpanelrefs,GATK4_GERMLINECNVCALLER,java,./git_repos/createpanelrefs/modules/nf-core/gatk4/germlinecnvcaller/main.nf,40,"gatk --java-options ""-Xmx${avail_mem}g -XX:-UsePerfData"" \\"
27377,createpanelrefs,GATK4_INTERVALLISTTOOLS,java,./git_repos/createpanelrefs/modules/nf-core/gatk4/intervallisttools/main.nf,34,"gatk --java-options ""-Xmx${avail_mem}M -XX:-UsePerfData"" \\"
27692,createpanelrefs,GATK4_FILTERINTERVALS,java,./git_repos/createpanelrefs/modules/nf-core/gatk4/filterintervals/main.nf,35,"gatk --java-options ""-Xmx${avail_mem}M -XX:-UsePerfData"" \\"
27823,createpanelrefs,GATK4_ANNOTATEINTERVALS,java,./git_repos/createpanelrefs/modules/nf-core/gatk4/annotateintervals/main.nf,43,"gatk --java-options ""-Xmx${avail_mem}M -XX:-UsePerfData"" \\"
27966,createpanelrefs,GATK4_COLLECTREADCOUNTS,java,./git_repos/createpanelrefs/modules/nf-core/gatk4/collectreadcounts/main.nf,40,"gatk --java-options ""-Xmx${avail_mem}M -XX:-UsePerfData"" \\"


In [194]:
len(df_operator_clustered)

4970

In [195]:
multiqc = operator_script[operator_script['operator'] == 'multiqc']
pd.set_option('display.max_colwidth', None)
multiqc

,pipeline,process_name,operator,file_path,line_number,line_content
1836,readsimulator,MULTIQC,multiqc,./git_repos/readsimulator/modules/nf-core/multiqc/main.nf,30,multiqc \\
1877,readsimulator,MULTIQC,multiqc,./git_repos/readsimulator/modules/nf-core/multiqc/main.nf,40,"multiqc: \$( multiqc --version | sed -e ""s/multiqc, version //g"" )"
1892,readsimulator,MULTIQC,multiqc,./git_repos/readsimulator/modules/nf-core/multiqc/main.nf,46,mkdir multiqc_data
1895,readsimulator,MULTIQC,multiqc,./git_repos/readsimulator/modules/nf-core/multiqc/main.nf,47,touch multiqc_plots
1903,readsimulator,MULTIQC,multiqc,./git_repos/readsimulator/modules/nf-core/multiqc/main.nf,48,touch multiqc_report.html
1928,readsimulator,MULTIQC,multiqc,./git_repos/readsimulator/modules/nf-core/multiqc/main.nf,52,"multiqc: \$( multiqc --version | sed -e ""s/multiqc, version //g"" )"
7325,callingcards,MULTIQC,multiqc,./git_repos/callingcards/modules/nf-core/multiqc/main.nf,30,multiqc \\
7366,callingcards,MULTIQC,multiqc,./git_repos/callingcards/modules/nf-core/multiqc/main.nf,40,"multiqc: \$( multiqc --version | sed -e ""s/multiqc, version //g"" )"
7380,callingcards,MULTIQC,multiqc,./git_repos/callingcards/modules/nf-core/multiqc/main.nf,46,touch multiqc_data
7383,callingcards,MULTIQC,multiqc,./git_repos/callingcards/modules/nf-core/multiqc/main.nf,47,touch multiqc_plots


In [196]:
df_operator_clustered

,operator,Assigned Group
0,{log},6
1,{output},6
2,samtools,0
3,echo,2
4,shell:,4
5,python,4
6,mkdir,4
7,&1,6
8,rm,3
9,bcftools,0


In [197]:
df_operator_clustered = df_operator_clustered.rename(columns={'Operator': 'operator'})

merged_df = pd.merge(operator_script, df_operator_clustered, on='operator')

In [198]:
merged_df

,pipeline,process_name,operator,file_path,line_number,line_content,Assigned Group
0,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,335,echo $workflow.manifest.version > v_pipeline.txt,2
1,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt,2
2,diaproteomics,statistical_post_processing,echo,./git_repos/diaproteomics/main.nf,971,"msstats.R > msstats.log || echo ""Optional MSstats step failed. Please check logs and re-run or do a manual statistical analysis.""",2
3,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,47,"[ -z ""\$INDEX"" ] && echo ""Bowtie2 index files not found"" 1>&2 && exit 1",2
4,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,68,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2
5,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,69,samtools: \$(echo \$(samtools --version 2>&1) | sed 's/^.*samtools //; s/Using.*\$//'),2
6,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,88,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2
7,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,89,samtools: \$(echo \$(samtools --version 2>&1) | sed 's/^.*samtools //; s/Using.*\$//'),2
8,readsimulator,BOWTIE2_BUILD,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/build/main.nf,27,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2
9,readsimulator,BOWTIE2_BUILD,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/build/main.nf,39,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2


We merge the biotool and the all operator df here

In [199]:
bioinformatics_script

NameError: name 'bioinformatics_script' is not defined

In [200]:
merged_df[merged_df["operator"]=="fastp"]

,pipeline,process_name,operator,file_path,line_number,line_content,Assigned Group
25669,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,39,fastp \\,6
25670,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,43,--json ${prefix}.fastp.json \\,6
25671,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,44,--html ${prefix}.fastp.html \\,6
25672,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,48,2> >(tee ${prefix}.fastp.log >&2) \\,6
25673,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,49,| gzip -c > ${prefix}.fastp.fastq.gz,6
25674,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,53,"fastp: \$(fastp --version 2>&1 | sed -e ""s/fastp //g"")",6
25675,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,60,fastp \\,6
25676,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,62,--out1 ${prefix}.fastp.fastq.gz \\,6
25677,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,64,--json ${prefix}.fastp.json \\,6
25678,detaxizer,FASTP,fastp,./git_repos/detaxizer/modules/nf-core/fastp/main.nf,65,--html ${prefix}.fastp.html \\,6


In [201]:
bioinformatics_script['operator']

NameError: name 'bioinformatics_script' is not defined

In [202]:
merged_df['operator']

0                           echo
1                           echo
2                           echo
3                           echo
4                           echo
5                           echo
6                           echo
7                           echo
8                           echo
9                           echo
10                          echo
11                          echo
12                          echo
13                          echo
14                          echo
15                          echo
16                          echo
17                          echo
18                          echo
19                          echo
20                          echo
21                          echo
22                          echo
23                          echo
24                          echo
                  ...           
29743                      2.bed
29744                      2.bed
29745                       $STR
29746                       chr/
29747     

In [203]:
new_rows = bioinformatics_script[~bioinformatics_script['operator'].isin(df_operator_clustered['operator'])]

NameError: name 'bioinformatics_script' is not defined

In [204]:
bioinformatics_script

NameError: name 'bioinformatics_script' is not defined

In [205]:
new_rows = bioinformatics_script[bioinformatics_script['operator'].isin(df_operator_clustered['operator'])]
new_rows

NameError: name 'bioinformatics_script' is not defined

In [206]:
# Concatenate df_operator_clustered and the filtered rows from bioinformatics_script
extended_df = pd.concat([df_operator_clustered, new_rows], ignore_index=True)

NameError: name 'new_rows' is not defined

In [207]:
df_operator_clustered

,operator,Assigned Group
0,{log},6
1,{output},6
2,samtools,0
3,echo,2
4,shell:,4
5,python,4
6,mkdir,4
7,&1,6
8,rm,3
9,bcftools,0


In [208]:
filtered_df = merged_df[merged_df['Assigned Group'] == 0]

bio_tools = filtered_df['operator'].value_counts()

print(bio_tools)

operator
fastq                              1166
samtools                           1098
fasta                              1021
fastq.gz                            645
fastqc                              236
bedtools                            154
bcftools                            125
bwa-mem2                             44
biom                                 40
$samtools                            37
seqtk                                26
fgbio                                24
seqkit                               20
fasta.gz                             18
bamtools                             14
gtf2bed                              12
biotype                              11
bedGraphToBigWig                     11
bamCoverage                          10
mmseqs                                9
{params.genome}                       8
transcripts.fasta                     6
bedClip                               4
complementBed                         3
sortBed                        

Discover the Assigned Groups

In [209]:
bioinformatic_tools = merged_df[merged_df['Assigned Group'] == 0]

In [210]:
count_operators = merged_df[merged_df['Assigned Group'] == 0].groupby('operator')['operator'].count()
nunique_pipeline = merged_df[merged_df['Assigned Group'] == 0].groupby('operator')['pipeline'].nunique()
nunique_file_path = merged_df[merged_df['Assigned Group'] == 0].groupby('operator')['file_path'].nunique()

bio_tools = pd.DataFrame({
    'pipeline': nunique_pipeline,
    'file_path': nunique_file_path,
    'sum_operators': count_operators
})

bio_tools = bio_tools.sort_values(by='pipeline', ascending=False)

In [211]:
bio_tools.head(60)

,pipeline,file_path,sum_operators
operator,,,
fasta,73,527,1021
fastq,72,241,1166
fastqc,67,71,236
samtools,58,356,1098
fastq.gz,57,132,645
$samtools,24,37,37
bedtools,21,58,154
gtf2bed,12,12,12
bcftools,12,39,125


In [212]:
merged_df['operator'][merged_df['Assigned Group'] == 1].value_counts()

operator
else                             232
done                             125
while                             70
classifier                        25
elif                              16
prefix=                           10
VALIDATION_STRINGENCY=LENIENT      8
NR==1                              7
centrifuge                         7
done;                              6
SnpSift                            4
LC_ALL=C                           4
ASSUME_SORTED=true                 3
sniffles                           3
SORT_ORDER=coordinate              2
centrifuge-kreport                 2
=NF;i++                            2
=NF;                               2
download-antismash-databases       2
VALIDATION_STRINGENCY=SILENT       2
REMOVE_DUPLICATES=TRUE             2
REMOVE_DUPLICATES=true             1
RGPL=illumina                      1
LC_COLLATE=C                       1
ORS=                               1
RGID=1                             1
FNR==1                       

In [213]:
merged_df['operator'][merged_df['Assigned Group'] == 2].value_counts()

operator
echo                        2515
params                       632
print                        575
printf                       468
grep                         439
head                         353
gawk                          80
MarkDuplicates                73
cutadapt                      69
paired                        62
plotFingerprint               25
fusioncatcher                 15
porechop                       9
mawk                           6
abricate                       3
zgrep                          3
cuteSV                         3
/head                          2
{params.aligner}               2
egrep                          2
{params.min_contig_size}       1
{params.dbsnp}                 1
{params.mills}                 1
deduplicate_bismark            1
DB=`echo                       1
Name: count, dtype: int64

In [214]:
merged_df['operator'][merged_df['Assigned Group'] == 3].value_counts()

operator
zcat                   13
sourmash               11
/bin/rm                 6
coverm                  3
MapRTTransformer        2
thermorawfileparser     2
Name: count, dtype: int64

In [215]:
merged_df['operator'][merged_df['Assigned Group'] == 4].value_counts()

operator
mkdir                            473
python                           415
sort                             374
script                           341
report                           273
export                           246
Rscript                          195
summary                          186
feature                          144
results                          138
source                           130
picard                           122
rename                           111
graph                             78
kraken2                           66
return                            64
trim_galore                       54
trimmed                           51
Filter                            51
histogram                         43
drop                              43
convert                           37
gffread                           29
featureCounts                     29
cram                              28
                                ... 
coverage2cytosine            

In [216]:
merged_df['operator'][merged_df['Assigned Group'] == 4].value_counts()

operator
mkdir                            473
python                           415
sort                             374
script                           341
report                           273
export                           246
Rscript                          195
summary                          186
feature                          144
results                          138
source                           130
picard                           122
rename                           111
graph                             78
kraken2                           66
return                            64
trim_galore                       54
trimmed                           51
Filter                            51
histogram                         43
drop                              43
convert                           37
gffread                           29
featureCounts                     29
cram                              28
                                ... 
coverage2cytosine            

In [217]:
pd.set_option("display.min_rows", 50)
merged_df['operator'][merged_df['Assigned Group'] == 0].value_counts()

operator
fastq                              1166
samtools                           1098
fasta                              1021
fastq.gz                            645
fastqc                              236
bedtools                            154
bcftools                            125
bwa-mem2                             44
biom                                 40
$samtools                            37
seqtk                                26
fgbio                                24
seqkit                               20
fasta.gz                             18
bamtools                             14
gtf2bed                              12
biotype                              11
bedGraphToBigWig                     11
bamCoverage                          10
mmseqs                                9
{params.genome}                       8
transcripts.fasta                     6
bedClip                               4
complementBed                         3
sortBed                        

In [218]:
merged_df['pipeline_process_name'] = merged_df['pipeline'] + "." + merged_df['process_name']

In [219]:
bio_tools = merged_df[merged_df['Assigned Group'] == 0]

bio_tools = bio_tools.groupby('operator').agg({'pipeline': 'nunique', 'pipeline_process_name': 'nunique'}).sort_values(by='pipeline', ascending=False)
bio_tools.sum()

pipeline                  482
pipeline_process_name    1831
dtype: int64

In [220]:
grouped_df = merged_df.groupby('Assigned Group').agg({'pipeline': 'nunique', 'pipeline_process_name': 'nunique', 'operator': 'count'}).reset_index()

In [221]:
sum_processes = 3081

In [222]:
merged_df

,pipeline,process_name,operator,file_path,line_number,line_content,Assigned Group,pipeline_process_name
0,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,335,echo $workflow.manifest.version > v_pipeline.txt,2,diaproteomics.get_software_versions
1,diaproteomics,get_software_versions,echo,./git_repos/diaproteomics/main.nf,336,echo $workflow.nextflow.version > v_nextflow.txt,2,diaproteomics.get_software_versions
2,diaproteomics,statistical_post_processing,echo,./git_repos/diaproteomics/main.nf,971,"msstats.R > msstats.log || echo ""Optional MSstats step failed. Please check logs and re-run or do a manual statistical analysis.""",2,diaproteomics.statistical_post_processing
3,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,47,"[ -z ""\$INDEX"" ] && echo ""Bowtie2 index files not found"" 1>&2 && exit 1",2,readsimulator.BOWTIE2_ALIGN
4,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,68,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2,readsimulator.BOWTIE2_ALIGN
5,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,69,samtools: \$(echo \$(samtools --version 2>&1) | sed 's/^.*samtools //; s/Using.*\$//'),2,readsimulator.BOWTIE2_ALIGN
6,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,88,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2,readsimulator.BOWTIE2_ALIGN
7,readsimulator,BOWTIE2_ALIGN,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/align/main.nf,89,samtools: \$(echo \$(samtools --version 2>&1) | sed 's/^.*samtools //; s/Using.*\$//'),2,readsimulator.BOWTIE2_ALIGN
8,readsimulator,BOWTIE2_BUILD,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/build/main.nf,27,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2,readsimulator.BOWTIE2_BUILD
9,readsimulator,BOWTIE2_BUILD,echo,./git_repos/readsimulator/modules/nf-core/bowtie2/build/main.nf,39,bowtie2: \$(echo \$(bowtie2 --version 2>&1) | sed 's/^.*bowtie2-align-s version //; s/ .*\$//'),2,readsimulator.BOWTIE2_BUILD


In [223]:
#sum_processes = merged_df['process_name'].nunique()
#sum_processes = merged_df['file_path'].nunique()

#6154 if-conditions


grouped_df['Ratio to total processes'] = grouped_df['pipeline_process_name']/sum_processes

In [224]:
grouped_df = grouped_df.rename(columns={'file_path': 'Files', 'operator' : 'Total operator occurrences'})

In [225]:
grouped_df

,Assigned Group,pipeline,pipeline_process_name,Total operator occurrences,Ratio to total processes
0,0,86,1308,4767,0.424537
1,1,73,281,543,0.091204
2,2,92,1948,5341,0.632262
3,3,20,27,37,0.008763
4,4,93,1620,4596,0.525803
5,5,68,265,776,0.086011
6,6,94,2396,13708,0.777670


In [ ]:
import os
import re
import csv

def find_operators_in_script(directory, output_file_name):
    result_list = []

    for pipeline in os.listdir(directory):
        subdirectory_path = os.path.join(directory, pipeline)
        folder_name = os.path.basename(subdirectory_path)
        print(pipeline)

        for root, dirs, files in os.walk(subdirectory_path):
            for filename in files:
                file_path = os.path.join(root, filename)
                
                if filename.endswith('.nf') or filename.endswith('.config'):
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                        lines = file.readlines()
                    
                    inside_block = False

                    for line_num, line in enumerate(lines):
                        stripped_line = line.strip()

                        if '"""' in stripped_line:
                            inside_block = not inside_block

                        if inside_block:
                            match = re.match(r'^(\w+):\s', stripped_line)
                            if match:
                                keyword = match.group(1)
                                result_list.append([pipeline, keyword, file_path, line_num+1, line.strip()])

    if result_list:
        write_to_csv(result_list, output_file_name)
    else:
        print("No results found.")

def write_to_csv(result_list, output_file_name):
    with open(f'./results/{output_file_name}.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['pipeline', 'keyword', 'file_path', 'line_number', 'line_content'])
        writer.writerows(result_list)

directory_path = './git_repos'
output_file_name = 'keyword_occurrences'

find_operators_in_script(directory_path, output_file_name)


In [68]:
import pandas as pd
df = pd.read_csv('./results/keyword_occurrences.csv')

#df.head(60)
df

,pipeline,keyword,file_path,line_number,line_content
0,airrflow,fastqc,./git_repos\airrflow\modules\local\fastqc_post...,28,"fastqc: \$( fastqc --version | sed -e ""s/FastQ..."
1,airrflow,igblastn,./git_repos\airrflow\modules\local\fetch_datab...,29,"igblastn: \$( igblastn -version | grep -o ""igb..."
2,airrflow,changeo,./git_repos\airrflow\modules\local\fetch_datab...,30,changeo: \$( AssignGenes.py --version | awk -F...
3,airrflow,gunzip,./git_repos\airrflow\modules\local\gunzip.nf,24,gunzip: \$(echo \$(gunzip --version 2>&1) | se...
4,airrflow,python,./git_repos\airrflow\modules\local\merge_UMI.nf,26,python: \$( echo \$(python --version | grep -o...
...,...,...,...,...,...
3793,viralrecon,tabix,./git_repos\viralrecon\modules\nf-core\tabix\t...,39,tabix: \$(echo \$(tabix -h 2>&1) | sed 's/^.*V...
3794,viralrecon,unicycler,./git_repos\viralrecon\modules\nf-core\unicycl...,43,unicycler: \$(echo \$(unicycler --version 2>&1...
3795,viralrecon,untar,./git_repos\viralrecon\modules\nf-core\untar\m...,48,untar: \$(echo \$(tar --version 2>&1) | sed 's...
3796,viralrecon,untar,./git_repos\viralrecon\modules\nf-core\untar\m...,60,untar: \$(echo \$(tar --version 2>&1) | sed 's...


In [69]:
len(df)
keyword_counts = df['keyword'].value_counts()
#pd.set_option("display.max_rows", 999)

len(keyword_counts)

491

In [70]:
result = df.groupby('keyword').agg({
    'pipeline': 'nunique',
    'file_path': 'count'  
}).sort_values(by='pipeline', ascending=False)
result

,pipeline,file_path
keyword,,
multiqc,66,130
python,56,182
fastqc,52,109
samtools,45,515
pigz,29,65
...,...,...
ITSx,1,1
IsobaricAnalyzer,1,1
ganon,1,6
